# groupby 메서드로 평균값 구하기

In [3]:
import pandas as pd
df=pd.read_csv('../data/gapminder.tsv',sep='\t')

In [2]:
avg_life_exp_by_year = df.groupby('year').lifeExp.mean()
print(avg_life_exp_by_year)

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64


# 분할-반영-결합 과정 살펴보기

In [9]:

#"분할과정"
##중복없이 year열의 데이터를 추출하기 unique() 메서드 활용
years = df.year.unique()
print(years)

#"반영과정"
y1952 = df.loc[df.year == 1952, :]
print(y1952.head())
y1952_mean = y1952.lifeExp.mean()
print(y1952_mean)


y1957 = df.loc[df.year == 1957, :]
y1957_mean = y1957.lifeExp.mean()
print(y1957_mean)

y1962 = df.loc[df.year == 1962, :]
y1962_mean = y1962.lifeExp.mean()
print(y1962_mean)

#...
y2007 = df.loc[df.year == 2007, :]
y2007_mean = y2007.lifeExp.mean()
print(y2007_mean)

#"결합과정"

df2 = pd.DataFrame({"year":[1952,1957,1962,2007], "":[y1952_mean, y1957_mean, y1962_mean, y2007_mean]})
print(df2)

[1952 1957 1962 1967 1972 1977 1982 1987 1992 1997 2002 2007]
        country continent  year  lifeExp       pop    gdpPercap
0   Afghanistan      Asia  1952   28.801   8425333   779.445314
12      Albania    Europe  1952   55.230   1282697  1601.056136
24      Algeria    Africa  1952   43.077   9279525  2449.008185
36       Angola    Africa  1952   30.015   4232095  3520.610273
48    Argentina  Americas  1952   62.485  17876956  5911.315053
49.05761971830987
51.507401126760534
53.60924901408449
67.00742253521126
   year           
0  1952  49.057620
1  1957  51.507401
2  1962  53.609249
3  2007  67.007423


# 평균값을 구하는 사용자 함수와 groupby 메서드

In [22]:
#평균값을 구하는 사용자 함수를 정의하여 agg메서드를 groupby메서드와 조화시키기

def my_mean(values):
    n = len(values)
    sum = 0
    for value in values:
        sum += value
    return sum / n

agg_my_mean = df.groupby('year').lifeExp.agg(my_mean)
print(agg_my_mean)

year
1952    49.057620
1957    51.507401
1962    53.609249
1967    55.678290
1972    57.647386
1977    59.570157
1982    61.533197
1987    63.212613
1992    64.160338
1997    65.014676
2002    65.694923
2007    67.007423
Name: lifeExp, dtype: float64


# 두 개의 인잣값을 받아 처리하는 사용자 함수와 groupby 메서드

In [24]:
def my_mean_diff(values, diff_value):
    n=len(values)
    sum = 0
    for value in values:
        sum += value
    mean = sum / n
    return mean - diff_value

global_mean = df.lifeExp.mean()
print(global_mean)

agg_mean_diff = df.groupby('year').lifeExp.agg(my_mean_diff, diff_value=global_mean)
print(agg_mean_diff)

59.47443936619714
year
1952   -10.416820
1957    -7.967038
1962    -5.865190
1967    -3.796150
1972    -1.827053
1977     0.095718
1982     2.058758
1987     3.738173
1992     4.685899
1997     5.540237
2002     6.220483
2007     7.532983
Name: lifeExp, dtype: float64


# 집계 메서드를 리스트, 딕셔너리에 담아 전달하기

In [26]:
#numpy 메서드를 활용하여 0이 아닌 값의 개수, 평균, 표준편차 구하기(리스트)

import numpy as np

gdf =df.groupby('year').lifeExp.agg([np.count_nonzero, np.mean, np.std])
print(gdf)

#평균과 중위수를 키와 값으로 표현하는 딕셔너리로 구하기

gdf_dict = df.groupby('year').agg({'lifeExp': 'mean', 'pop':'median','gdpPercap':'median'})
print(gdf_dict)

      count_nonzero       mean        std
year                                     
1952            142  49.057620  12.225956
1957            142  51.507401  12.231286
1962            142  53.609249  12.097245
1967            142  55.678290  11.718858
1972            142  57.647386  11.381953
1977            142  59.570157  11.227229
1982            142  61.533197  10.770618
1987            142  63.212613  10.556285
1992            142  64.160338  11.227380
1997            142  65.014676  11.559439
2002            142  65.694923  12.279823
2007            142  67.007423  12.073021
        lifeExp         pop    gdpPercap
year                                    
1952  49.057620   3943953.0  1968.528344
1957  51.507401   4282942.0  2173.220291
1962  53.609249   4686039.5  2335.439533
1967  55.678290   5170175.5  2678.334740
1972  57.647386   5877996.5  3339.129407
1977  59.570157   6404036.5  3798.609244
1982  61.533197   7007320.0  4216.228428
1987  63.212613   7774861.5  4280.300366
19

# 표준 점수 계산하기

In [6]:
def my_zscore(x):
    return (x-x.mean()) / x.std()

transform_z = df.groupby('year').lifeExp.transform(my_zscore)

print(transform_z.head())

#데이터의 크기 비교

print(df.shape)

print(transform_z.shape)

#데이터의 행 개수가 줄어들지 않았다.

0   -1.656854
1   -1.731249
2   -1.786543
3   -1.848157
4   -1.894173
Name: lifeExp, dtype: float64
(1704, 6)
(1704,)


# 누락값을 평균값으로 처리하기

In [7]:
import seaborn as sns
import numpy as np

np.random.seed(42)
tips_10 = sns.load_dataset('tips').sample(10)
tips_10.loc[np.random.permutation(tips_10.index)[:4], 'total_bill'] = np.NaN
print(tips_10)

count_sex = tips_10.groupby('sex').count()
print(count_sex)

def fill_na_mean(x):
    avg = x.mean()
    return x.fillna(avg)

total_bill_group_mean = tips_10.groupby('sex').total_bill.transform(fill_na_mean)
tips_10['fill_total_bill'] = total_bill_group_mean
print(tips_10)

     total_bill   tip     sex smoker   day    time  size
24        19.82  3.18    Male     No   Sat  Dinner     2
6          8.77  2.00    Male     No   Sun  Dinner     2
153         NaN  2.00    Male     No   Sun  Dinner     4
211         NaN  5.16    Male    Yes   Sat  Dinner     4
198         NaN  2.00  Female    Yes  Thur   Lunch     2
176         NaN  2.00    Male    Yes   Sun  Dinner     2
192       28.44  2.56    Male    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
9         14.78  3.23    Male     No   Sun  Dinner     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2
        total_bill  tip  smoker  day  time  size
sex                                             
Male             4    7       7    7     7     7
Female           2    3       3    3     3     3
     total_bill   tip     sex smoker   day    time  size  fill_total_bill
24        19.82  3.18    Male     No   Sat  Dinner     2          19.8200
6          8.77  2.00    Male

# 데이터 필터링 사용하기 ─ filter 메서드

In [8]:
tips = sns.load_dataset('tips')
print(tips.shape)

print(tips['size'].value_counts())

#filter 메서드를 활용하여 30번 이상의 주문을 받은 테이블을 추려내기

tips_filtered = tips.groupby('size').filter(lambda x: x['size'].count() >= 30)
print(tips_filtered.shape)
print(tips_filtered['size'].value_counts())

(244, 7)
2    156
3     38
4     37
5      5
1      4
6      4
Name: size, dtype: int64
(231, 7)
2    156
3     38
4     37
Name: size, dtype: int64


# 그룹 오브젝트 저장하여 살펴보기

In [9]:
tips_10 = sns.load_dataset('tips').sample(10, random_state=42)
print(tips_10)

grouped = tips_10.groupby('sex')
print(grouped)

print(grouped.groups)

     total_bill   tip     sex smoker   day    time  size
24        19.82  3.18    Male     No   Sat  Dinner     2
6          8.77  2.00    Male     No   Sun  Dinner     2
153       24.55  2.00    Male     No   Sun  Dinner     4
211       25.89  5.16    Male    Yes   Sat  Dinner     4
198       13.00  2.00  Female    Yes  Thur   Lunch     2
176       17.89  2.00    Male    Yes   Sun  Dinner     2
192       28.44  2.56    Male    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
9         14.78  3.23    Male     No   Sun  Dinner     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2
{'Male': [24, 6, 153, 211, 176, 192, 9], 'Female': [198, 124, 101]}


# 그룹 오브젝트의 평균 구하기

In [25]:
avgs = grouped.mean()
print(avgs)

print(tips_10.columns)

        total_bill       tip      size
sex                                   
Male         20.02  2.875714  2.571429
Female       13.62  2.506667  2.000000
Index(['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size'], dtype='object')


# 그룹 오브젝트에서 데이터 추출하고 반복하기

In [34]:
female = grouped.get_group('Female')
print(female)

for sex_group in grouped:
    print(sex_group)

     total_bill   tip     sex smoker   day    time  size
198       13.00  2.00  Female    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2
('Male',      total_bill   tip   sex smoker   day    time  size
24        19.82  3.18  Male     No   Sat  Dinner     2
6          8.77  2.00  Male     No   Sun  Dinner     2
153       24.55  2.00  Male     No   Sun  Dinner     4
211       25.89  5.16  Male    Yes   Sat  Dinner     4
176       17.89  2.00  Male    Yes   Sun  Dinner     2
192       28.44  2.56  Male    Yes  Thur   Lunch     2
9         14.78  3.23  Male     No   Sun  Dinner     2)
('Female',      total_bill   tip     sex smoker   day    time  size
198       13.00  2.00  Female    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2)


In [31]:
for sex_group in grouped:
    print('the type is: {}\n'.format(type(sex_group)))
    print('the length is: {}\n'.format(len(sex_group)))
    first_element = sex_group[0]
    print('the first element is: {}\n'.format(first_element))
    print('it has a type of: {}\n'.format(type(sex_group[0])))
    second_element = sex_group[1]
    print('it has a type of: {}\n'.format(type(second_element)))
    
    print('what we have:')
    print(sex_group)
    
    break

the type is: <class 'tuple'>

the length is: 2

the first element is: Male

it has a type of: <class 'str'>

it has a type of: <class 'pandas.core.frame.DataFrame'>

what we have:
('Male',      total_bill   tip   sex smoker   day    time  size
24        19.82  3.18  Male     No   Sat  Dinner     2
6          8.77  2.00  Male     No   Sun  Dinner     2
153       24.55  2.00  Male     No   Sun  Dinner     4
211       25.89  5.16  Male    Yes   Sat  Dinner     4
176       17.89  2.00  Male    Yes   Sun  Dinner     2
192       28.44  2.56  Male    Yes  Thur   Lunch     2
9         14.78  3.23  Male     No   Sun  Dinner     2)


# 그룹 오브젝트 계산하고 살펴보기

In [11]:
bill_sex_time = tips_10.groupby(['sex','time'])
group_avg = bill_sex_time.mean()

print(group_avg)

               total_bill       tip      size
sex    time                                  
Male   Lunch    28.440000  2.560000  2.000000
       Dinner   18.616667  2.928333  2.666667
Female Lunch    12.740000  2.260000  2.000000
       Dinner   15.380000  3.000000  2.000000


In [15]:
print(type(group_avg))
print(group_avg.columns)
print(group_avg.index)

<class 'pandas.core.frame.DataFrame'>
Index(['total_bill', 'tip', 'size'], dtype='object')
MultiIndex([(  'Male',  'Lunch'),
            (  'Male', 'Dinner'),
            ('Female',  'Lunch'),
            ('Female', 'Dinner')],
           names=['sex', 'time'])


In [17]:
group_method = tips_10.groupby(['sex','time']).mean().reset_index()
print(group_method)
group_param = tips_10.groupby(['sex','time'], as_index=False).mean()
print(group_param)

      sex    time  total_bill       tip      size
0    Male   Lunch   28.440000  2.560000  2.000000
1    Male  Dinner   18.616667  2.928333  2.666667
2  Female   Lunch   12.740000  2.260000  2.000000
3  Female  Dinner   15.380000  3.000000  2.000000
      sex    time  total_bill       tip      size
0    Male   Lunch   28.440000  2.560000  2.000000
1    Male  Dinner   18.616667  2.928333  2.666667
2  Female   Lunch   12.740000  2.260000  2.000000
3  Female  Dinner   15.380000  3.000000  2.000000
